In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

import collections
INF = 1 << 30
class MCMF:
  def __init__(self, V, src, snk) :
    self.len = V
    self.G = [[] for _ in range(V)]
    self.C = [[0] * V for _ in range(V)]
    self.F = [[0] * V for _ in range(V)]
    self.W = [[0] * V for _ in range(V)]
    self.chk = [False] * V
    self.vis = [False] * V
    self.lvl = [0] * V
    self.src = src
    self.snk = snk
    self.mc = 0

  def addEdge(self, u, v, c, w) :
    self.G[u].append(v)
    self.G[v].append(u)
    self.C[u][v] = c
    self.W[u][v] = w
    self.W[v][u] = -w

  def bfs(self) :
    Q = collections.deque([self.src])
    inQ = [False] * self.len 
    lvl = [INF] * self.len
    lvl[self.src] = 0

    while Q :
      u = Q.popleft()
      inQ[u] = False

      for v in self.G[u] :
        if self.C[u][v] > self.F[u][v] and lvl[v] > lvl[u] + self.W[u][v] :
          lvl[v] = lvl[u] + self.W[u][v]
          if not inQ[v] :
            Q.append(v)
            inQ[v] = True
    
    self.lvl = lvl
    return self.lvl[self.snk] != INF

  def dfs(self, u, w) :
    if u == self.snk : return w
    flow = w
    self.vis[u] = True

    for v in self.G[u] :
      if not flow : break
      if not self.vis[v] and not self.chk[v] and self.C[u][v] - self.F[u][v] > 0 and self.lvl[v] == self.lvl[u] + self.W[u][v] :
        res = self.dfs(v, min(flow, self.C[u][v] - self.F[u][v]))
        if res == 0 : 
          self.chk[v] = True
          continue
        self.F[u][v] += res
        self.F[v][u] -= res
        self.mc += res * self.W[u][v]
        flow -= res
    
    self.vis[u] = False
    return w - flow

  def calc(self) :
    mf = 0
    while self.bfs() :
      self.chk = [False] * self.len
      self.vis = [False] * self.len
      mf += self.dfs(self.src, INF)
    return self.mc, mf

ccw = lambda o, a, b: (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])
def cross(a, b) :
  t1 = ccw(a[0], a[1], b[0]) * ccw(a[0], a[1], b[1])
  t2 = ccw(b[0], b[1], a[0]) * ccw(b[0], b[1], a[1])
  if t1 < 0 and t2 < 0 : return True
  if t1 == 0 and t2 == 0: return b[0] <= a[1] and a[0] <= b[1]
  return t1 <= 0 and t2 <= 0

def sol() :
  n, m = map(int, input().split())
  L1 = [tuple(map(int, input().split())) for _ in range(n)]
  L2 = [tuple(map(int, input().split())) for _ in range(m)]

  G = [[0] * m for _ in range(n)]
  for i, (x1, y1, x2, y2, w1) in enumerate(L1) :
    for j, (x3, y3, x4, y4, w2) in enumerate(L2) :
      if cross(((x1, y1), (x2, y2)), ((x3, y3), (x4, y4))) :
        G[i][j] = w1 * w2
  
  V = n + m + 2
  src, snk = V - 2, V - 1
  mcmf = MCMF(V, src, snk)
  for i in range(n) :
    mcmf.addEdge(src, i, 1, 0)
  for i in range(m) :
    mcmf.addEdge(i + n, snk, 1, 0)
  
  for i in range(n) :
    for j in range(m) :
      if G[i][j] :
        mcmf.addEdge(i, j + n, 1, -G[i][j])
  
  mc, mf = mcmf.calc()
  sys.stdout.write(f'{mf} {-mc}\n')

for _ in range(int(input())) :
  sol()

### 자력솔
- 각 선분이 교차할때 유량 간선이 생기게 되며, 교차한 선분들의 무게를 곱한 만큼의 유량의 비용이 된다. 벌써 유량 그래프 뚝딱만들었다.